<h1>Data set : Flight delays & cancellations</h1>

<span>DEV 2 : EFREI 2022-2023</span>
<ul>
<li>Amine</li>
<li>Fabian</li>
</ul>

In [530]:
#Import Libraries
import numpy as np 
from pyspark.sql.functions import *
from pyspark.ml.feature import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
import pyspark.ml.evaluation as evalu

In [531]:
#Configuration Jobs

spark = SparkSession \
    .builder \
    .appName("Flights") \
    .getOrCreate()

spark.conf.set('spark.sql.repl.eagerEval.enabled',True)

<h2>Load csv Data from 3 sources to DataFrame</h2>

In [532]:
#Flights
flightsDF= spark.read.option("header","true").csv("./Data/flights.csv")
#Airports
airportsDF= spark.read.option("header","true").csv("./Data/airports.csv")
#Airline
airlinesDF= spark.read.option("header","true").csv("./Data/airlines.csv")

<h2>Functions transform or PreProcessing</h2>

In [533]:
#Functions to transform or remove columns
def RemoveUnnecessaryColumnsFunction(df: DataFrame) -> DataFrame:
    columns_to_drop = [
                        'ORIGIN_AIRPORT',
                        'IATA_CODE',
                        'AIRPORT',
                        'CITY',
                        'STATE',
                        'COUNTRY',
                        'LATITUDE',
                        'LONGITUDE']
    return df.drop(*columns_to_drop)

<h2>All to 1 DataFrame FlightsDF instead of 3</h2>

In [534]:
#join airlines to flights to avoid code
flightsDF = flightsDF.join(airlinesDF,flightsDF.AIRLINE ==  airlinesDF.IATA_CODE,"inner").drop(flightsDF.AIRLINE).drop(col("IATA_CODE")).withColumnRenamed("AIRLINE","AIRLINE_") 
flightsDF = flightsDF.join(airportsDF,flightsDF.ORIGIN_AIRPORT ==  airportsDF.IATA_CODE,"inner").withColumnRenamed("CITY","CITY_DEPART_") 
flightsDF = RemoveUnnecessaryColumnsFunction(flightsDF)
flightsDF = flightsDF.join(airportsDF,flightsDF.DESTINATION_AIRPORT ==  airportsDF.IATA_CODE,"inner").withColumnRenamed("CITY","CITY_ARRIVAL_").drop(col("DESTINATION_AIRPORT")) 
flightsDF = RemoveUnnecessaryColumnsFunction(flightsDF)

<h2>Data</h2>

In [535]:
#Transform Date
#flightsDF = flightsDF.withColumn('DATE_', to_date(concat_ws('-',flightsDF.YEAR,flightsDF.MONTH,flightsDF.DAY))).drop(*['YEAR','MONTH','DAY','DAY_OF_WEEK'])

In [536]:
print("Count Flights : ",flightsDF.count()," Count columns:", len(flightsDF.columns))
print("Flights numbers : ",flightsDF.select('FLIGHT_NUMBER').distinct().count())  # 6327 num de vol unique
#print(flightsDF.groupBy('FLIGHT_NUMBER').count().orderBy('count', ascending=False))
flightsDF.groupBy(*['CITY_DEPART_', 'CITY_ARRIVAL_']).count().orderBy('count', ascending=False)

Count Flights :  306160  Count columns: 31
Flights numbers :  6327


CITY_DEPART_,CITY_ARRIVAL_,count
New York,Chicago,760
Chicago,New York,753
Los Angeles,New York,744
New York,Los Angeles,743
San Francisco,Los Angeles,713
Los Angeles,San Francisco,707
Boston,New York,706
New York,Boston,703
Ft. Lauderdale,New York,622
New York,Ft. Lauderdale,620


<h1>Flights canceled</h1>

In [537]:
canceledFlight = flightsDF.filter(flightsDF.AIR_TIME.isNull())
print("Canceled flight : " , canceledFlight.count());

Canceled flight :  6424


<h2>Flights not canceled </h2>

In [538]:
notCanceledFlight = flightsDF.filter(flightsDF.ARRIVAL_DELAY.isNotNull() & flightsDF.AIR_TIME.isNotNull() & flightsDF.AIRLINE_.isNotNull() &  flightsDF.TAIL_NUMBER.isNotNull())
print("Not canceled flight : " , notCanceledFlight.count());
notCanceledFlight.select(notCanceledFlight["ARRIVAL_DELAY"])

Not canceled flight :  299736


ARRIVAL_DELAY
-22
-9
5
-9
-21
8
-17
-10
-13
-15


<h1>Transform & Create Indexer & OneHotEncoder</h1>

In [539]:
#Transform
#  supported types
notCanceledFlight = notCanceledFlight.withColumn("MONTH", notCanceledFlight.MONTH.cast("integer"))
notCanceledFlight = notCanceledFlight.withColumn("DAY_OF_WEEK", notCanceledFlight.DAY_OF_WEEK.cast("integer"))
notCanceledFlight = notCanceledFlight.withColumn("AIR_TIME", notCanceledFlight.AIR_TIME.cast("integer"))
notCanceledFlight = notCanceledFlight.withColumn("DISTANCE", notCanceledFlight.DISTANCE.cast("double"))
notCanceledFlight = notCanceledFlight.withColumn("ARRIVAL_DELAY", notCanceledFlight.ARRIVAL_DELAY.cast("integer"))

#create nex isLate column

# Create is_late
notCanceledFlight = notCanceledFlight.withColumn("IS_LATE_", notCanceledFlight.ARRIVAL_DELAY > 0)
notCanceledFlight.limit(10)
print('is Late Flight',notCanceledFlight.groupBy('IS_LATE_').count())
notCanceledFlight = notCanceledFlight.withColumnRenamed("IS_LATE_", 'label')
notCanceledFlight = notCanceledFlight.withColumn("label", notCanceledFlight.label.cast("integer"))



airline_indexer = StringIndexer(inputCol="AIRLINE_", outputCol="AIRLINE_index")
airline_encoder = OneHotEncoder(inputCol="AIRLINE_index", outputCol="AIRLINE_fact")

dest_indexer = StringIndexer(inputCol="CITY_ARRIVAL_", outputCol="CITY_ARRIVAL_index")
dest_encoder = OneHotEncoder(inputCol="CITY_ARRIVAL_index", outputCol="CITY_ARRIVAL_fact")

tail_indexer = StringIndexer(inputCol="TAIL_NUMBER", outputCol="TAIL_NUMBER_index")
tail_encoder = OneHotEncoder(inputCol="TAIL_NUMBER_index", outputCol="TAIL_NUMBER_fact")

is Late Flight +--------+------+
|IS_LATE_| count|
+--------+------+
|    true|131710|
|   false|168026|
+--------+------+



<h1>VectorAssembler</h1>

In [540]:
vec_assembler = VectorAssembler(inputCols=[
    "MONTH", 
    "DAY_OF_WEEK", 
    "AIR_TIME",
    "DISTANCE",
    "AIRLINE_fact", 
    "CITY_ARRIVAL_fact", 
    "TAIL_NUMBER_fact"
], outputCol="features")

<h1>PipeLine  & transform data again</h1>

In [541]:
flights_pipe = Pipeline(stages=[dest_indexer, dest_encoder, airline_indexer, airline_encoder, tail_indexer, tail_encoder, vec_assembler])

piped_data = flights_pipe.fit(notCanceledFlight).transform(notCanceledFlight)

train_data, test_data = piped_data.randomSplit([.7, .3])

print('data points(rows) in train data :',  train_data.count())
print('data points(rows) in test data :',  test_data.count())


data points(rows) in train data : 209837
data points(rows) in test data : 89899


<h1>Trainning data</h1>

In [542]:
# Create a LogisticRegression Estimator
lr = LogisticRegression()
evaluator = evalu.BinaryClassificationEvaluator(metricName="areaUnderROC")

LogisticRegressionModel = lr.fit(train_data)

test_results = LogisticRegressionModel.transform(test_data)
print(evaluator.evaluate(test_results))


0.6218575807200716
